In [ ]:
#GPU TEST
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

In [25]:
# LOAD JSON FILE

# IMPORTS
from os import path, mkdir
import json
import requests

# FILENAME
name = 'memes-prettified.json' if path.exists('memes-prettified.json') else 'memes.json'

# DOWNLOAD
mkdir('./images')
with open(name, 'r') as json_file:
    load = json.load(json_file)
    for meme in load:
        mkdir('images/'+str(meme['id']))
        for generated_meme in meme['generated_memes']:
            print('Name: ' + meme['name'] + ' ID: ' + str(meme['id']),'Downloading ID ' + str(generated_meme['id']))
            with open('./images/'+str(meme['id'])+'/'+str(generated_meme['id'])+'.jpg', 'wb') as f:
                f.write(requests.get('https:'+str(generated_meme['image_url'])).content)



Bling ID: 0 Downloading ID 1491
Name: Drake Hotline Bling ID: 0 Downloading ID 1492
Name: Drake Hotline Bling ID: 0 Downloading ID 1493
Name: Drake Hotline Bling ID: 0 Downloading ID 1494
Name: Drake Hotline Bling ID: 0 Downloading ID 1495
Name: Drake Hotline Bling ID: 0 Downloading ID 1496
Name: Drake Hotline Bling ID: 0 Downloading ID 1497
Name: Drake Hotline Bling ID: 0 Downloading ID 1498
Name: Drake Hotline Bling ID: 0 Downloading ID 1499
Name: Drake Hotline Bling ID: 0 Downloading ID 1500
Name: Drake Hotline Bling ID: 0 Downloading ID 1501
Name: Drake Hotline Bling ID: 0 Downloading ID 1502
Name: Drake Hotline Bling ID: 0 Downloading ID 1503
Name: Drake Hotline Bling ID: 0 Downloading ID 1504
Name: Drake Hotline Bling ID: 0 Downloading ID 1505
Name: Drake Hotline Bling ID: 0 Downloading ID 1506
Name: Drake Hotline Bling ID: 0 Downloading ID 1507
Name: Drake Hotline Bling ID: 0 Downloading ID 1508
Name: Drake Hotline Bling ID: 0 Downloading ID 1509
Name: Drake Hotline Bling ID: 0 

KeyboardInterrupt: 